In [4]:
import numpy as np
import cv2 as cv
MARGIN = 10  # pixels
ROW_SIZE = 10  # pixels
FONT_SIZE = 1
FONT_THICKNESS = 1
TEXT_COLOR = (255, 0, 0)  # red

def visualize(
    image,
    detection_result
) -> np.ndarray:
  """Draws bounding boxes on the input image and return it.
  Args:
    image: The input RGB image.
    detection_result: The list of all "Detection" entities to be visualize.
  Returns:
    Image with bounding boxes.
  """
  for detection in detection_result.detections:
    # Draw bounding_box
    bbox = detection.bounding_box
    start_point = bbox.origin_x, bbox.origin_y
    end_point = bbox.origin_x + bbox.width, bbox.origin_y + bbox.height
    cv.rectangle(image, start_point, end_point, TEXT_COLOR, 3)

    # Draw label and score
    category = detection.categories[0]
    category_name = category.category_name
    probability = round(category.score, 2)
    result_text = category_name + ' (' + str(probability) + ')'
    text_location = (MARGIN + bbox.origin_x,
                     MARGIN + ROW_SIZE + bbox.origin_y)
    cv.putText(image, result_text, text_location, cv.FONT_HERSHEY_PLAIN,
                FONT_SIZE, TEXT_COLOR, FONT_THICKNESS)

  return image

In [32]:
import cv2 as cv
from matplotlib import pyplot as plt
import numpy as np
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

video = cv.VideoCapture(0)
while(True):
    ret, frame = video.read()
    if ret == True:
        frame = cv.flip(frame, 1)
        cv.imshow("Salida", frame)
        base_options = python.BaseOptions(model_asset_path='efficientdet.tflite')
        
        options = vision.ObjectDetectorOptions(base_options=base_options,
                                       score_threshold=0.5)
        
        detector = vision.ObjectDetector.create_from_options(options)

        image = mp.Image(
        image_format=mp.ImageFormat.SRGB,
        data=frame)

        detection_result = detector.detect(image)
        image_copy = np.copy(image.numpy_view())
        annotated_image = visualize(image_copy, detection_result)
        cv.imshow("Salida2", annotated_image)
    if ret == False:
        video.set(cv.CAP_PROP_POS_FRAMES, 0)
    if cv.waitKey(10) & 0xFF == 27: break

video.release()
cv.destroyAllWindows()